In [2]:
import dgl
import torch

In [4]:
# Create a heterograph with 3 node type and 3 edge type
graph_data = {
    ('drug','interacts','drug'): (torch.tensor([0,1]), torch.tensor([1,2])),
    ('drug','interacts','gene'): (torch.tensor([0,1]), torch.tensor([2,3])),
    ('drug','treats','disease'): (torch.tensor([1]), torch.tensor([2]))
}
# 이 코드의 뜻은 drug node type이 interacts라는 엣지타입을 가지고 drug 노드 타입과 연결된것
# drug node type이 interacts라는 엣지타입을 가지고 gene 노드 타입과 연결된것
# drug node type이 treats 엣지 타입을 가지고 disease 노드 타입과 연결된 것을 나타낸다.
g = dgl.heterograph(graph_data)
print(g.ntypes)
print(g.etypes)
print(g.canonical_etypes)

['disease', 'drug', 'gene']
['interacts', 'interacts', 'treats']
[('drug', 'interacts', 'drug'), ('drug', 'interacts', 'gene'), ('drug', 'treats', 'disease')]


In [5]:
#즉 일반적으로 먼저 공부하는 homogeneous graph와 bipartite graph는 엣지 관계가 1개인 heterogeneous graph이다
print(g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [12]:
#노드와 엣지가 여러 타입이 사용되는 경우에는 타입을 명시해줘야지 정보를 얻을 수 있다.
print(g.num_nodes())
print(g.num_nodes('drug'))
# print(g.nodes()) -> error
print(g.nodes('drug'))
print(g.nodes('gene'))
print(g.nodes('disease'))

10
3
tensor([0, 1, 2])
tensor([0, 1, 2, 3])
tensor([0, 1, 2])


In [13]:
#node or edge type에 대한 features 설정하기
g.nodes['drug'].data['dx'] = torch.ones(3,1)
print(g.nodes['drug'].data['dx'])

g.edges['treats'].data['ex'] = torch.zeros(1,1)
print(g.edges['treats'].data['ex'])

tensor([[1.],
        [1.],
        [1.]])
tensor([[0.]])


In [16]:
# heterogeneous graph에서 message passing은 각 관계에 대한 메세지 전달 
# 각 노드 타입에 대한 모든 집계 결과를 합치는 축약으로 이루어져있다.
# 그러면 본격적으로 실습에 들어가보자

import numpy as np
n_users = 1000 #user 1000명
n_items = 500 #item 500개
n_follows = 3000 #follows 3000
n_clicks = 5000 #클릭데이터 5000개
n_dislikes = 500 #싫어요 500개
n_hetero_features = 10
n_user_classes = 5 # user label
n_max_clicks = 10 # clik label

follow_src = np.random.randint(0,n_users, n_follows)
follow_dst = np.random.randint(0,n_users, n_follows)
click_src = np.random.randint(0,n_users,n_clicks) #user idx에서 click 어디에 했는지 즉 5000개
click_dst = np.random.randint(0,n_items,n_clicks) #위에 user에서 어떤 아이템인지
dislike_src = np.random.randint(0,n_users,n_dislikes)
dislike_dst = np.random.randint(0,n_items,n_dislikes)
hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

print(hetero_graph)



Graph(num_nodes={'item': 500, 'user': 1000},
      num_edges={('item', 'clicked-by', 'user'): 5000, ('item', 'disliked-by', 'user'): 500, ('user', 'click', 'item'): 5000, ('user', 'dislike', 'item'): 500, ('user', 'follow', 'user'): 3000, ('user', 'followed-by', 'user'): 3000},
      metagraph=[('item', 'user', 'clicked-by'), ('item', 'user', 'disliked-by'), ('user', 'item', 'click'), ('user', 'item', 'dislike'), ('user', 'user', 'follow'), ('user', 'user', 'followed-by')])


In [18]:
#heterogeneous graph의 각 노드 feature와 user의 클래스(노드분류), click edge의 라벨을 설정해준다.
hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users,n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items,n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0,n_user_classes,(n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1,n_max_clicks,(n_clicks,)).float()
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

In [19]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
# heterogeneous graph를 처리할 수 있는 모델 정의
# 각 관계별로 따로 처리해야되기 때문에 rel_names인자로 받는다.
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [21]:
#위의 모델에서 dgl.nn.HeteroGraphConv 는 노드의 타입들과 노드 피쳐 텐서를 받아서 각 타입의 변화한 값을 리턴해준다.
model = RGCN(n_hetero_features,32,n_user_classes,hetero_graph.etypes)
#마지막의 hetero_graph.etypes를 넣어줌으로서 관계별로 학습할 수 있다.
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']

node_features = {'user':user_feats,'item':item_feats}
# 1회 foward process했을 때를 보여준다
h_dict = model(hetero_graph,node_features)
h_user = h_dict['user']
h_item = h_dict['item']
print(h_dict)

{'item': tensor([[ 0.4085, -0.3442, -0.2905, -0.3642, -0.4166],
        [ 0.4111, -0.4494, -0.0868, -0.3961,  0.1084],
        [ 0.2562, -0.2515, -0.1196, -0.6014,  0.1129],
        ...,
        [ 0.4015, -0.2884, -0.8815,  0.2217,  0.4844],
        [ 0.0602, -0.2743,  0.0055, -0.0050,  0.1593],
        [ 0.1573, -0.4735, -0.1770, -0.2468,  0.2824]], grad_fn=<SumBackward1>), 'user': tensor([[ 9.1826e-01, -1.0417e-01,  5.0676e-01, -8.6102e-01, -6.0579e-01],
        [ 3.1130e-01, -7.8602e-02,  8.4745e-01, -1.2497e-01,  3.9142e-01],
        [ 6.6147e-01,  3.7854e-01,  3.9091e-01, -6.6784e-04, -8.7747e-02],
        ...,
        [ 6.4095e-01, -6.5442e-01, -3.9604e-01,  6.6888e-01, -1.3066e-01],
        [ 6.6090e-01, -3.9749e-01,  7.9074e-01,  4.4164e-03, -9.9020e-02],
        [ 1.1888e-01, -2.8105e-01,  1.2480e+00,  3.4606e-01, -4.8740e-01]],
       grad_fn=<SumBackward1>)}


In [25]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
epochs = 50
for i in range(epochs):
    model.train()
    logits = model(hetero_graph,node_features)['user']
    loss = F.cross_entropy(logits[train_mask],labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

#학습방법은 일반적인것과 같고 logits에 user를 써놓은 이유는 user의 클래스에만 관심이 있기 때문이다.


tensor(1.4209, grad_fn=<NllLossBackward0>)
tensor(1.4051, grad_fn=<NllLossBackward0>)
tensor(1.3703, grad_fn=<NllLossBackward0>)
tensor(1.3402, grad_fn=<NllLossBackward0>)
tensor(1.3213, grad_fn=<NllLossBackward0>)
tensor(1.2975, grad_fn=<NllLossBackward0>)
tensor(1.2694, grad_fn=<NllLossBackward0>)
tensor(1.2451, grad_fn=<NllLossBackward0>)
tensor(1.2247, grad_fn=<NllLossBackward0>)
tensor(1.2020, grad_fn=<NllLossBackward0>)
tensor(1.1769, grad_fn=<NllLossBackward0>)
tensor(1.1540, grad_fn=<NllLossBackward0>)
tensor(1.1337, grad_fn=<NllLossBackward0>)
tensor(1.1120, grad_fn=<NllLossBackward0>)
tensor(1.0885, grad_fn=<NllLossBackward0>)
tensor(1.0664, grad_fn=<NllLossBackward0>)
tensor(1.0458, grad_fn=<NllLossBackward0>)
tensor(1.0246, grad_fn=<NllLossBackward0>)
tensor(1.0025, grad_fn=<NllLossBackward0>)
tensor(0.9817, grad_fn=<NllLossBackward0>)
tensor(0.9619, grad_fn=<NllLossBackward0>)
tensor(0.9414, grad_fn=<NllLossBackward0>)
tensor(0.9206, grad_fn=<NllLossBackward0>)
tensor(0.90